# Case Study

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from google.colab import files

In [2]:
from google.colab import drive
drive.mount("/content/gDrive")

Mounted at /content/gDrive


In [3]:
HEADERS = {'User-Agent':
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
        AppleWebKit/537.36 (KHTML, like Gecko) \
        Chrome/90.0.4430.212 Safari/537.36'}

COOKIES = {}

In [4]:
def get_data(url:str):
    page = requests.get(url, cookies=COOKIES, headers=HEADERS)
    return page

In [5]:
def asin_number(soup):
    data_asins = []
    for item in soup.find_all("div", {"data-component-type": "s-search-result"}):
        data_asins.append(item['data-asin'])
    return data_asins

In [6]:
def fetch_href(soup):
    links = []
    for item in soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"}):
        links.append(item['href'])
    return links[0]

In [7]:
def customer_review(soup):
    data_str = ""
    td_tag_list = soup.findAll(lambda tag: "data-hook" in tag.attrs and tag["data-hook"] == "review")
    for item in td_tag_list:
        selected_item = item.findAll("span", attrs={'class': 'a-size-base review-text review-text-content'})
        if selected_item:
            temp_text = selected_item[0].get_text()
            if len(temp_text) > 0:
                data_str = data_str + temp_text
            else:
                data_str = data_str + " \n"
        else:
            data_str = data_str + " \n"
    result = data_str.split("\n")
    return (result)

In [8]:
def customer_rating(soup):
    data_out_list = []
    data_str = ""
    td_tag_list = soup.findAll(lambda tag: "data-hook" in tag.attrs and tag["data-hook"] == "review-star-rating")
    for item in td_tag_list:
        selected_item = item.findAll("span", attrs={'class': 'a-icon-alt'})
        if selected_item:
            data_str = selected_item[0].get_text().split("out")[0].strip(" ")
            data_int = int(float(data_str))
            data_out_list.append(data_int)
    return data_out_list

In [12]:
url = f"https://www.amazon.in/dp/B0C718TYDX"

response = get_data(url)
soup = BeautifulSoup(response.content)
link = fetch_href(soup)

In [13]:
link

'/Zemic-Portable-Travel-Umbrella-Umbrellas/product-reviews/B0C718TYDX?reviewerType=all_reviews'

In [14]:
url = f"https://www.amazon.in{link}&pageNumber=1"
response = get_data(url)
soup = BeautifulSoup(response.text)
review_data = customer_review(soup)

In [16]:
review_data = [review for review in review_data if len(review) > 0]
temp_rating_data = customer_rating(soup)

In [27]:
all_reviews = []
rating_data = []
all_reviews += review_data
reviews_df = pd.DataFrame(data=all_reviews,columns=['reviews'])
rating_df=pd.DataFrame(data=temp_rating_data,columns=['ratings'])

In [33]:
final_df=pd.merge(reviews_df,rating_df, left_index=True, right_index=True)
final_df

,reviews,ratings
0,Nice and strong,4
1,I recently purchased an umbrella from a reputa...,5
2,Bought it in emergency but turns out it's a gr...,4
3,quality to be verified after rain.,4
4,Size is small for an adult .. the overall qual...,4
5,This Umbrella is lightweight and comes with a ...,4
6,It is very difficult to close the umbrella,4
7,Loks sturdy. But one of the ribs was already o...,4
8,Fit for 2person and it's handy and lightweight...,4
9,Good quality product.,4
